<font size="5">Section 3: Panel data</font>

## Housekeeping and Data

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
import scipy 
import linearmodels.panel as lmp

%matplotlib inline

We use data from the ENIA firm panel survey. Variable description follows:  


- *ID*: firm unique identifier  
- *year*: survey year  
- *tamano*: 1 large, 2 medium, 3 small, 4 micro  
- *sales*: sales (in log of 1,000 CLP)  
- *age*: firm age at time of survey  
- *foreign*: non-domestic firm (binary)  
- *export*: production for export (binary)  
- *workers*: log of number of workers  
- *fomento*: firm receives public incentives (binary)  
- *iyd*: firm does I+D (binary)  
- *impuestos*: taxes (in million US)  
- *utilidades*: firm revenue (in million US)  

For purposes of this analysis we will consider tamano as a continous value (moving from largest to smallest as number increases).

In [141]:
# enia data
enia = pd.read_csv('../data/enia.csv')
enia.dropna(inplace=True)
enia.reset_index(drop=True, inplace=True)

#variable construction
X=enia[['age','sales','fomento','export','tamano','impuestos']]
Xm=(X.groupby(enia['ID']).transform('mean'))
Xid=enia[['ID','year','workers','age','sales','fomento','export','tamano','impuestos']]
Xc=pd.DataFrame(np.c_[Xid, Xm], columns=['ID','year','workers','age','sales','fomento','export','tamano','impuestos','mage','msales','mfomento','mexport','mtamano','mimpuestos'])

#set panel structure
Xc = Xc.set_index(["ID","year"])
Xc.describe()

,workers,age,sales,fomento,export,tamano,impuestos,mage,msales,mfomento,mexport,mtamano,mimpuestos
count,39104.000000,39104.000000,39104.000000,39104.000000,39104.000000,39104.000000,39104.000000,39104.000000,39104.000000,39104.000000,39104.000000,39104.000000,39104.000000
mean,1.757726,15.305084,3.574172,0.076105,0.111191,2.248773,0.203856,15.305084,3.574172,0.076105,0.111191,2.248773,0.203856
std,1.186507,12.488330,1.692742,0.265169,0.314372,1.153089,15.869466,11.409701,1.484028,0.210185,0.287169,1.133066,9.065384
min,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-180.992528,0.000000,0.000000,0.000000,0.000000,1.000000,-60.319504
25%,0.778151,7.000000,2.337643,0.000000,0.000000,1.000000,0.000000,7.000000,2.490218,0.000000,0.000000,1.000000,0.000000
50%,1.785330,14.000000,3.553321,0.000000,0.000000,2.000000,0.000007,14.000000,3.673942,0.000000,0.000000,2.000000,0.000024
75%,2.661813,20.000000,4.539098,0.000000,0.000000,3.000000,0.000167,20.250000,4.587509,0.000000,0.000000,3.000000,0.001182
max,5.845915,190.000000,10.309005,1.000000,1.000000,4.000000,2981.494528,157.000000,9.638881,1.000000,1.000000,4.000000,995.749564


## Pooled OLS

We can use statsmodels to estimate a simple OLS regression to explain the average drivers of the firm's demand for workers (Pooled OLS in panel data).

In [142]:
y=Xc['workers']
X=Xc[['age','sales','fomento','export','tamano','impuestos']]
X=sm.add_constant(X)

model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                workers   R-squared:                       0.509
Model:                            OLS   Adj. R-squared:                  0.509
Method:                 Least Squares   F-statistic:                     6745.
Date:                Tue, 27 Sep 2022   Prob (F-statistic):               0.00
Time:                        16:50:40   Log-Likelihood:                -48280.
No. Observations:               39104   AIC:                         9.657e+04
Df Residuals:                   39097   BIC:                         9.663e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.2938      0.022    150.362      0.0

In [143]:
model=lmp.PooledOLS(y,X)
OLS=model.fit(cov_type="robust")
print(OLS)

                          PooledOLS Estimation Summary                          
Dep. Variable:                workers   R-squared:                        0.5086
Estimator:                  PooledOLS   R-squared (Between):              0.5474
No. Observations:               39104   R-squared (Within):               0.0917
Date:                Tue, Sep 27 2022   R-squared (Overall):              0.5086
Time:                        16:50:44   Log-likelihood                -4.828e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      6745.2
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                 F(6,39097)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             7415.2
                            

## First differences 

In [144]:
X=Xc[['age','sales','fomento','export','tamano','impuestos']]
model=lmp.FirstDifferenceOLS(y,X)
fd=model.fit(cov_type="robust")
print(fd)

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\shared\utility.py:187: FutureWarning: In a future version of pandas all arguments of MultiIndex.set_levels except for the argument 'levels' will be keyword-only
  df.index = df.index.set_levels(final_levels, [0, 1])


                     FirstDifferenceOLS Estimation Summary                      
Dep. Variable:                workers   R-squared:                        0.2001
Estimator:         FirstDifferenceOLS   R-squared (Between):             -1.3115
No. Observations:               14188   R-squared (Within):               0.2040
Date:                Tue, Sep 27 2022   R-squared (Overall):             -1.1216
Time:                        16:50:48   Log-likelihood                -1.448e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      591.42
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                 F(6,14182)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             381.83
                            

## Fixed Effects 

In [145]:
X=Xc[['age','sales','fomento','export','tamano','impuestos']]
X=sm.add_constant(X)
model=lmp.PanelOLS(y,X, entity_effects=True)
fe=model.fit(cov_type="robust")
print(fe)

                          PanelOLS Estimation Summary                           
Dep. Variable:                workers   R-squared:                        0.2255
Estimator:                   PanelOLS   R-squared (Between):              0.2717
No. Observations:               39104   R-squared (Within):               0.2255
Date:                Tue, Sep 27 2022   R-squared (Overall):              0.2902
Time:                        16:50:58   Log-likelihood                -1.435e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      726.34
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                 F(6,14970)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             332.31
                            

## Random Effects


In [146]:
model=lmp.RandomEffects(y,X)
re=model.fit(cov_type="robust")
print(re)

                        RandomEffects Estimation Summary                        
Dep. Variable:                workers   R-squared:                        0.3878
Estimator:              RandomEffects   R-squared (Between):              0.5399
No. Observations:               39104   R-squared (Within):               0.1514
Date:                Tue, Sep 27 2022   R-squared (Overall):              0.5017
Time:                        16:51:05   Log-likelihood                -3.453e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      4127.1
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                 F(6,39097)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             5240.3
                            

In [147]:
re.variance_decomposition

Effects                   0.311160
Residual                  0.318662
Percent due to Effects    0.494045
Name: Variance Decomposition, dtype: float64

## Model comparison

In [148]:
print(lmp.compare({"FE": fe, "RE": re, "Pooled": OLS}))

                           Model Comparison                           
                                    FE                RE        Pooled
----------------------------------------------------------------------
Dep. Variable                  workers           workers       workers
Estimator                     PanelOLS     RandomEffects     PooledOLS
No. Observations                 39104             39104         39104
Cov. Est.                       Robust            Robust        Robust
R-squared                       0.2255            0.3878        0.5086
R-Squared (Within)              0.2255            0.1514        0.0917
R-Squared (Between)             0.2717            0.5399        0.5474
R-Squared (Overall)             0.2902            0.5017        0.5086
F-statistic                     726.34            4127.1        6745.2
P-value (F-stat)                0.0000            0.0000        0.0000
=====================     ============   ===============   ===========
const 

C:\ProgramData\Anaconda3\lib\site-packages\linearmodels\panel\results.py:1022: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  1,


In [149]:
import numpy.linalg as la
from scipy import stats

def hausman(fe, re):
 diff = fe.params-re.params
 psi = fe.cov - re.cov
 dof = diff.size -1
 W = diff.dot(la.inv(psi)).dot(diff)
 pval = stats.chi2.sf(W, dof)
 return W, dof, pval

In [150]:
htest = hausman(fe, re) 
print("Hausman Test: chi-2 = {0}, df = {1}, p-value = {2}".format(htest[0], htest[1], htest[2]))

Hausman Test: chi-2 = 489.87847525273105, df = 6, p-value = 1.2731322706600914e-102


## Correlated Random Effects

In [155]:
X=Xc[['age','sales','fomento','export','tamano','impuestos','mage','msales','mfomento','mexport','mtamano','mimpuestos']]
X=sm.add_constant(X)
model=lmp.RandomEffects(y,X)
cre=model.fit(cov_type="robust")
print(cre)

                        RandomEffects Estimation Summary                        
Dep. Variable:                workers   R-squared:                        0.4211
Estimator:              RandomEffects   R-squared (Between):              0.5540
No. Observations:               39104   R-squared (Within):               0.2255
Date:                Tue, Sep 27 2022   R-squared (Overall):              0.5278
Time:                        16:53:07   Log-likelihood                -3.343e+04
Cov. Estimator:                Robust                                           
                                        F-statistic:                      2369.9
Entities:                       24128   P-value                           0.0000
Avg Obs:                       1.6207   Distribution:                F(12,39091)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             3144.8
                            

In [156]:
print(lmp.compare({"FE": fe, "RE": re, "CRE": cre}))

                             Model Comparison                             
                                    FE                RE               CRE
--------------------------------------------------------------------------
Dep. Variable                  workers           workers           workers
Estimator                     PanelOLS     RandomEffects     RandomEffects
No. Observations                 39104             39104             39104
Cov. Est.                       Robust            Robust            Robust
R-squared                       0.2255            0.3878            0.4211
R-Squared (Within)              0.2255            0.1514            0.2255
R-Squared (Between)             0.2717            0.5399            0.5540
R-Squared (Overall)             0.2902            0.5017            0.5278
F-statistic                     726.34            4127.1            2369.9
P-value (F-stat)                0.0000            0.0000            0.0000
=====================    

<font size="3">**Tarea 2**</font>

<u> *Instrucciones* </u>

Los resultados de los ejericicios propuestos se deben entregar como un notebook por correo electronico a *juan.caro@uni.lu* el dia 3/10 hasta las 21:00. 

Es importante considerar que el código debe poder ejecutarse en cualquier computadora con la data original del repositorio. Recordar la convencion para el nombre de archivo ademas de incluir en su documento titulos y encabezados por seccion. La data a utilizar es **charls.csv**.

Las variables tienen la siguiente descripcion:

- INID: identificador unico
- wave: periodo de la encuesta (1-3)
- cesd: puntaje en la escala de salud mental (0-30)
- child: numero de hijos
- drinkly: bebio alcohol en el ultimo mes (binario)
- hrsusu: horas promedio trabajo semanal
- hsize: tamano del hogar
- intmonth: mes en que fue encuestado/a (1-12)
- married: si esta casado/a (binario)
- retired: si esta pensionado/a (binario)
- schadj: años de escolaridad
- urban: zona urbana (binario)
- wealth: riqueza neta (miles RMB)
- age: edad al entrar a la encuesta (no varia entre periodos)

Preguntas:

1. Cargar la base de datos *charls.csv* en el ambiente. Identifique los tipos de datos que se encuentran en la base, realice estadisticas descriptivas sobre las variables importantes (Hint: Revisar la distribuciones, datos faltantes, outliers, etc.) y limpie las variables cuando sea necesario. 

2. Ejecute un modelo Pooled OLS para explicar el puntaje en la escala de salud mental (CESD). Seleccione las variables dependientes a incluir en el modelo final e interprete su significado. 

3. Ejecute un modelo de efectos fijos para explicar el puntaje en la escala de salud mental (CESD).  Seleccione las variables dependientes a incluir en el modelo final e interprete su significado. 

4. Ejecute un modelo de efectos aleatorios para explicar el puntaje en la escala de salud mental (CESD). Seleccione las variables dependientes a incluir en el modelo final e interprete su significado. 

5. Comente los resultados obtenidos en 2, 3 y 4. ¿Cuáles y por qué existen las diferencias entre los resultados?. En su opinión, ¿Cuál sería el más adecuado para responder la pregunta de investgación y por qué? ¿Qué variables resultaron ser robustas a la especificación?

6. Ejecute un modelo de efectos aleatorios correlacionados (CRE) para explicar el puntaje en la escala de salud mental (CESD). Seleccione las variables dependientes a incluir en el modelo final e interprete su significado. Es este modelo adecuado, dada la data disponible, para modelar el componente no observado?

7. Usando el modelo CRE, prediga la distribucion del componente no observado. Que puede inferir respecto de la heterogeneidad fija en el tiempo y su impacto en el puntaje CESD? 

8. Usando sus respuestas anteriores, que modelo prefiere? que se puede inferir en general respecto del efecto de las variables explicativas sobre el puntaje CESD?
